In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
# 题目一：根据日期和订单字段，要求得出周次（week）、订单总和、日均订单、极大值订单、极小值订单字段
# 读取数据集
df1 = pd.read_excel('./示例文件.xlsx',sheet_name='data1')
df1.head()

,日期,订单,Unnamed: 2
0,2021-03-01,757,9
1,2021-03-02,997,9
2,2021-03-03,956,9
3,2021-03-04,830,9
4,2021-03-05,122,9


In [3]:
df1['周次'] = df1['日期'].dt.isocalendar().week.map(str)
df1

,日期,订单,Unnamed: 2,周次
0,2021-03-01,757,9,9
1,2021-03-02,997,9,9
2,2021-03-03,956,9,9
3,2021-03-04,830,9,9
4,2021-03-05,122,9,9
5,2021-03-06,406,9,9
6,2021-03-07,222,9,9
7,2021-03-08,516,10,10
8,2021-03-09,748,10,10
9,2021-03-10,229,10,10


In [4]:
df1_result = df1.groupby(['周次'])['订单'].agg(['sum', 'mean', 'max','min']).round(0).reset_index().sort_values(by=['周次'])
df1_result

,周次,sum,mean,max,min
0,10,4128,590.0,798,229
1,11,4162,595.0,948,192
2,12,3418,488.0,871,164
3,13,1264,421.0,789,135
4,9,4290,613.0,997,122


In [5]:
# 题目二：行转列并求每个人的平均分和总分
# 读取数据集
df2 = pd.read_excel('./示例文件.xlsx',sheet_name='data2')
df2.head()

,姓名,科目,分数
0,A,语文,65
1,B,语文,86
2,A,数学,94
3,B,数学,63
4,A,外语,67


In [6]:
df2_pv1 = pd.pivot_table(df2,index=['姓名'],columns=['科目'],values=['分数'],aggfunc=[np.sum],margins=True,margins_name='总分',fill_value=0)
df2_pv2 = pd.pivot_table(df2,index=['姓名'],values=['分数'],aggfunc=[np.mean],fill_value=0)
df2_result = pd.merge(df2_pv1,df2_pv2,on='姓名',how='left').reset_index()
df2_result

D:\software\anaconda\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (3 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,姓名,"(sum, 分数, 外语)","(sum, 分数, 数学)","(sum, 分数, 语文)","(sum, 分数, 总分)","(mean, 分数)"
0,A,67,94,65,226,75.333333
1,B,72,63,86,221,73.666667
2,总分,139,157,151,447,NaN


In [7]:
# 题目三：按照 区 排序，看到每一个业务区内订单排名前 3 名的门店及订单数
# 读取数据集
df3 = pd.read_excel('./示例文件.xlsx',sheet_name='data3')
df3.head()

,省,市,区,店铺名称,订单数
0,A,A,A,A,2033
1,A,A,B,B,1729
2,B,D,D,C,4030
3,B,D,E,D,2648
4,B,D,D,E,2714


In [8]:
df3_group = df3.groupby(['区','店铺名称'])['订单数'].sum().reset_index()
df3_group['组内降序排名'] = df3.groupby(['区'])['订单数'].rank(method='dense',ascending=False).astype(int)
df3_result = df3_group.query('组内降序排名 <= 3')
df3_result

,区,店铺名称,订单数,组内降序排名
0,A,A,2033,1
1,B,B,1729,1
2,C,F,1950,1
3,D,C,4030,1
4,D,E,2714,2
5,E,D,2648,1
6,E,G,2233,2
7,F,H,4025,1
